
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://moodle.df.senac.br/faculdade/pluginfile.php/1/theme_lambda/logo/1716924091/Logo-SENAC-PNG.png" style="width: 300px; height: 150px">
</div>

#Usando Spark SQL para tratamento de dados



## Limpando Dados

À medida que inspecionamos e limpamos nossos dados, precisaremos construir várias expressões e consultas para realizar transformações nosso conjunto de dados.

Expressões são construídas a partir de colunas, operadores e funções existentes. Elas podem ser usados ​​em instruções **`SELECT`** para expressar transformações que criam novas colunas.

Muitos comandos de consulta SQL padrão (por exemplo, **`DISTINCT`**, **`WHERE`**, **`GROUP BY`**, etc.) estão disponíveis no Spark SQL para expressar essas transformações.


## Overview dos dados

Iremos trabalhar com dados _raw_ de usuários, no seguinte esquema:

| campo | tipo | descrição |
|---|---|---|
| user_id | string | unique identifier |
| user_first_touch_timestamp | long | momento em que o registro foi criado. Está em epoch. |
| email | string | endereço de e-mail mais recente que o usuário informou ao completar uma ação |
| updated | timestamp | horário em que o registro foi atualizado |
> O campo updated terá o valor da hora atual (`current_timestamp()`)

Vamos começar criando uma _temp view_ chamada _users_raw_ para esses dados que estão em formato parquet.

> Para ler arquivos parquet, podemos usar:

`%sql`

`SELECT * FROM read_files(
  'caminho',
  format => 'parquet'
  )`

ou

`%python`

`spark.read.parquet("caminho")`

**Dica:** podemos criar _temp views_ pelo PySpark ao ler um arquivo: `spark.read.parquet('caminho').createOrReplaceTempView('nome_view')`. E para apagá-la: `spark.catalog.dropTempView('nome_view')`


In [0]:
%sql
CREATE OR REPLACE TEMP VIEW users_raw() AS
SELECT *, current_timestamp() AS update FROM read_files("dbfs:/Volumes/senac/default/users",
 format => 'parquet', schemaEvolutionMode => 'none');

SELECT * FROM users_raw;

user_id,user_first_touch_timestamp,email,update
UA000000107338110,1593874163848994,blackburnjohn@gay.biz,2024-07-12T22:47:02.313+0000
UA000000107354520,1593876102569545,reedjennifer@freeman.com,2024-07-12T22:47:02.313+0000
UA000000107359655,1593876696866893,roberthubbard55@hotmail.com,2024-07-12T22:47:02.313+0000
UA000000107362264,1593876996344195,amandafrazier@hotmail.com,2024-07-12T22:47:02.313+0000
UA000000107362440,1593877015690335,laura65@garcia.biz,2024-07-12T22:47:02.313+0000
UA000000107363722,1593877154338249,tmiller67@yahoo.com,2024-07-12T22:47:02.313+0000
UA000000107367850,1593877599590013,xmartinez27@gmail.com,2024-07-12T22:47:02.313+0000
UA000000107370290,1593877869468334,uflynn@anderson.com,2024-07-12T22:47:02.313+0000
UA000000107371743,1593878036347579,nancyellis34@hotmail.com,2024-07-12T22:47:02.313+0000
UA000000107371836,1593878046932741,nporter32@gmail.com,2024-07-12T22:47:02.313+0000



## Inspecionando os dados

Primeiramente, vamos procurar por valores nulos/faltantes (_missing values_) no nosso conjunto de dados:


In [0]:
%sql
SELECT count(*), count(user_id), count(user_first_touch_timestamp), count(email), count(update)
FROM users_raw

count(1),count(user_id),count(user_first_touch_timestamp),count(email),count(update)
983,983,983,138,983



Baseado nas contagens acima, percebe-se que há uma série de valores nulos em uma das nossas colunas: email.

**Obs.:** Valores nulos não são considerados em funções de coluna/agregação (**`sum, count, max, min, avg,` etc**) e não são computados em funções matemáticas, como a soma (`a + b`)

- **`count(col)`** desconsidera o valor **`NULL`** quando conta colunas específicas ou expressões.
- **`count(*)`** é um caso especial, pois conta o total de linhas, independente do campo ser nulo ou não.

Pode-se contar valores nulos em um campo filtrando por linhas em que o campo seja `null`:

>**`count_if(col IS NULL)`** ou **`count(*)`** com um filtro para a cláusula **`where col IS NULL`**. 


In [0]:
%sql
SELECT count(*)
FROM users_raw
WHERE email IS NULL;

count(1)
845


In [0]:
%sql
select count(*) qtd from users_raw WHERE email IS NULL;

select count_if(email is null) from users_raw;

count_if((email IS NULL))
845


 
## Deduplicar registros/linhas
Podemos usar o **`DISTINCT *`** para remover linhas onde todas as colunas estão duplicadas


In [0]:
%sql
SELECT DISTINCT(*) FROM users_raw

user_id,user_first_touch_timestamp,email,update
UA000000107386620,1593879668454372,null,2024-07-12T23:00:55.877+0000
UA000000107388952,1593879927821763,null,2024-07-12T23:00:55.877+0000
UA000000107390788,1593880121354219,null,2024-07-12T23:00:55.877+0000
UA000000107387871,1593879811686726,null,2024-07-12T23:00:55.877+0000
UA000000107390859,1593880129935464,null,2024-07-12T23:00:55.877+0000
UA000000107385379,1593879536983818,null,2024-07-12T23:00:55.877+0000
UA000000107387703,1593879792795228,null,2024-07-12T23:00:55.877+0000
UA000000107392855,1593880342116334,nicholssandra@hotmail.com,2024-07-12T23:00:55.877+0000
UA000000107394461,1593880511685871,null,2024-07-12T23:00:55.877+0000
UA000000107396111,1593880678252189,null,2024-07-12T23:00:55.877+0000


In [0]:
usersDF = spark.read.table("users_raw")
usersDF.distinct().display()

user_id,user_first_touch_timestamp,email,update
UA000000107385910,1593879591306851,null,2024-07-12T23:01:08.872+0000
UA000000107385467,1593879545852333,null,2024-07-12T23:01:08.872+0000
UA000000107387751,1593879798315168,null,2024-07-12T23:01:08.872+0000
UA000000107395136,1593880580520931,null,2024-07-12T23:01:08.872+0000
UA000000107389284,1593879965734466,null,2024-07-12T23:01:08.872+0000
UA000000107386620,1593879668454372,null,2024-07-12T23:01:08.872+0000
UA000000107367850,1593877599590013,xmartinez27@gmail.com,2024-07-12T23:01:08.872+0000
UA000000107392926,1593880349970004,null,2024-07-12T23:01:08.872+0000
UA000000107390211,1593880064425772,null,2024-07-12T23:01:08.872+0000
UA000000107393878,1593880452156698,null,2024-07-12T23:01:08.872+0000



## Deduplicate registros com base em coluas específicas

Podemos deduplicas linhas com base em colunas específicas utilizando o **`GROUP BY`**, 
combinado com funções de coluna/agregação.

### Atividade

Utilizando a função de agregação **`max`** e os campos **user_id** e **user_first_touch_timestamp** com chaves:
- Traga os registros deduplicados, aplicando a função de agregação nas colunas restantes
- Salve os registros em uma _temp view_ chamada `users_deduplicado`


In [0]:
%sql
CREATE OR REPLACE TEMP VIEW users_deduplicado AS (
SELECT user_id, user_first_touch_timestamp, MAX(email) AS email, MAX(update) AS update
FROM users_raw
GROUP BY user_id, user_first_touch_timestamp);

SELECT * FROM users_deduplicado;

user_id,user_first_touch_timestamp,email,update
UA000000107335605,1593873851950592,danielcarrillo@smith.com,2024-07-12T23:19:24.643+0000
UA000000107338110,1593874163848994,blackburnjohn@gay.biz,2024-07-12T23:19:24.643+0000
UA000000107342625,1593874719224880,xjoseph@miller.biz,2024-07-12T23:19:24.643+0000
UA000000107345540,1593875054653990,theresahuber@yahoo.com,2024-07-12T23:19:24.643+0000
UA000000107347151,1593875250452082,melissamcdaniel@gmail.com,2024-07-12T23:19:24.643+0000
UA000000107349198,1593875490748942,hwhite@hall.com,2024-07-12T23:19:24.643+0000
UA000000107351411,1593875742437934,castrodavid@valdez.com,2024-07-12T23:19:24.643+0000
UA000000107354520,1593876102569545,reedjennifer@freeman.com,2024-07-12T23:19:24.643+0000
UA000000107355987,1593876269167805,robert03@cohen-kaiser.net,2024-07-12T23:19:24.643+0000
UA000000107356825,1593876365676183,sullivanjohn@dunn.com,2024-07-12T23:19:24.643+0000


## Validando o resultado

Vamos confirmar se os registros foram mesmo deduplicados

### Atividade

Verifique se os dados foram deduplicados. Há algumas formas para validar os dados:
- `having count(*) > 1`
- `max(row_count) <= 1`



In [0]:
%sql
SELECT max(row_count) <= 1 from(
SELECT user_id, user_first_touch_timestamp, email, update, count(*) as row_count
FROM users_deduplicado
GROUP BY user_id, user_first_touch_timestamp, email, update);


(max(row_count) <= 1)
true


In [0]:
%sql
SELECT user_id, user_first_touch_timestamp, email, update
FROM users_deduplicado
GROUP BY user_id, user_first_touch_timestamp, email, update
HAVING count(*) > 1;

user_id,user_first_touch_timestamp,email,update



## Formato de data e Regex

Agora que eliminamos os registros duplicados, vamos extrair mais dados dos nossos dados.

### Atividade
#### Utilizando funções de conversão de datas e expressões regulares:
- Converta o campo **`user_first_touch_timestamp`** para um campo chamado _first_touch_date_, apenas com a data no formato **yyyy-mm-dd**
- Converta o campo **`user_first_touch_timestamp`** para um campo chamado _first_touch_time_, apenas com a data no formato **HH:mm:ss**
- Extraia o domínio do e-mail para um campo chamado _email_domain_

**Dicas:**
- Antes de utilizar um campo _epoch_ como _timestamp_, faça a conversão `CAST(campo / 1e6 AS timestamp)`
- Para conversão de data utilize a função `date_format()`.
- Para expressão regular: regexp_extract(), padrão **"(?<=@).+"**

In [0]:
%sql
SELECT date_format(CAST(user_first_touch_timestamp/ 1e6 AS TIMESTAMP), 'yyyy-MM-dd') as first_touch_date,
       date_format(CAST(user_first_touch_timestamp/ 1e6 AS TIMESTAMP), 'HH:mm:ss') as first_touch_time,
        regexp_extract(email, "(?<=@).+",0) AS email_domain
FROM users_deduplicado;

first_touch_date,first_touch_time,email_domain
2020-07-04,14:44:11,smith.com
2020-07-04,14:49:23,gay.biz
2020-07-04,14:58:39,miller.biz
2020-07-04,15:04:14,yahoo.com
2020-07-04,15:07:30,gmail.com
2020-07-04,15:11:30,hall.com
2020-07-04,15:15:42,valdez.com
2020-07-04,15:21:42,freeman.com
2020-07-04,15:24:29,cohen-kaiser.net
2020-07-04,15:26:05,dunn.com



## Data Profile 

A partir do Databricks 9.1 há dois metodos para _data profiling_ a partir dos Notebooks: pela saída/resultado da célula e pela biblioteca `dbutils`.

Quando se trabalha com _data frames_ ou resultados de consultas SQL, você pdoe acessar uma guia dedicada ao **Data Profile**. Ao clicar nessa guia, é iniciado um extensivo processo de criação de _data profile_, gerando não apenas estatísticas sumarizadas, mas também histogramas que cobre todo o conjunto de dados, assegurando uma compreensiva visão dos dados, além do que está visível de forma tabular.

Esse _data profile_ abrange uma variedade de insights, incluindo informações sobre colunas numéricas, de texto e de data, tornando-o uma ferramenta poderosa para exploração e compreensão de dados.

**Usando pela resultado da célula:**

1. Na saída da célular, ao lado de _Table_, clique em `+`.

1. Então clique em **Data Profile**